# Pre-processing Behavioral data
### Inspiration from Ronny Eichler pre-processing code


In [1]:
# Importation 
import numpy as np
import base64
import struct
from cobs import cobs
from collections import namedtuple
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot
from tqdm.notebook import tqdm
import scipy.signal
from scipy.io import savemat
import rawpy
import imageio
import os
import pickle

#### Name variables

In [2]:
Path_non_decoded_files= '/home/melisamc/Documentos/photon2_testing/data/log_files/'
Path_decoded ='/home/melisamc/Documentos/photon2_testing/logfiles_decoded/'
path_output = '/home/melisamc/Documentos/photon2_testing/timestamps/'
Name='20220513-165536_677'

#### Path for file to decode

In [3]:
bp=os.path.join(Path_non_decoded_files, Name+'.b64')

## Decoding

<div class="alert alert-block alert-info">
<b>Info:</b> If the data packet change ( Possible with different version of Teensy) change format here 
</div>

In [4]:
# Format package
DataPacketDesc = {'type': 'B',
                  'size': 'B',
                  'crc16': 'H',
                  'packetID': 'I',
                  'us_start': 'I',
                  'us_end': 'I',
                  'analog': '8H',
                  'states': '8l',
                  'digitalIn': 'H',
                  'digitalOut': 'B',
                  'padding': 'x'}


DataPacket = namedtuple('DataPacket', DataPacketDesc.keys())
DataPacketStruct = '<' + ''.join(DataPacketDesc.values())
DataPacketSize = struct.calcsize(DataPacketStruct)

# package with non-digital data
dtype_no_digital = [
    ('type', np.uint8),
    ('size', np.uint8),
    ('crc16', np.uint16),
    ('packetID', np.uint32),
    ('us_start', np.uint32),
    ('us_end', np.uint32),
    ('analog', np.uint16, (8, )),
    ('states', np.float32, (8, ))]

# DigitalIn and DigitalOut
dtype_w_digital = dtype_no_digital + [('digital_in', np.uint16, (16, )), ('digital_out', np.uint8, (8, ))]

# Creating arrat with all the data (differenciation digital/non digital)
np_DataPacketType_noDigital = np.dtype(dtype_no_digital)
np_DataPacketType_withDigital = np.dtype(dtype_w_digital)

In [5]:
# function to count the packet number
def count_lines(fp):
    def _make_gen(reader):
        b = reader(2**16)
        while b:
            yield b
            b = reader(2**16)
    with open(fp, 'rb') as f:
        count = sum(buf.count(b'\n') for buf in _make_gen(f.raw.read))
    return count

In [6]:
%time num_lines = count_lines(bp)
log_duration = num_lines/1000/60
print(f'{num_lines} packets, ~{log_duration:0.2f} minutes')

CPU times: user 202 ms, sys: 36.3 ms, total: 239 ms
Wall time: 238 ms
2085888 packets, ~34.76 minutes


In [7]:
# Unpack the data as done on the teensy commander code
def unpack_data_packet(dp):
    s = struct.unpack(DataPacketStruct, dp)
    up = DataPacket(type=s[0], size=s[1], crc16=s[2], packetID=s[3], us_start=s[4], us_end=s[5],
                        analog=s[6:14], states=s[14:22], digitalIn=s[22], digitalOut=s[23], padding=None)

    return up

In [8]:
# Decode and create new dataset
data = np.zeros(num_lines, dtype=np_DataPacketType_withDigital)
non_digital_names = list(np_DataPacketType_noDigital.names)

with open(bp, 'rb') as bf:
    for nline, line in enumerate(tqdm(bf, total=num_lines)):
        bl = cobs.decode(base64.b64decode(line[:-1])[:-1])
        dp = unpack_data_packet(bl)

        data[non_digital_names][nline] = np.frombuffer(bl[:-4], dtype=np_DataPacketType_noDigital)
        digital_arr = np.frombuffer(bl[-4:], dtype=np.uint8)
        data[nline]['digital_in'] = np.hstack([np.unpackbits(digital_arr[0]), np.unpackbits(digital_arr[1])])
        data[nline]['digital_out'] = np.unpackbits(np.array(digital_arr[2], dtype=np.uint8))

  0%|          | 0/2085888 [00:00<?, ?it/s]

In [9]:
#Check for packetID jumps
jumps = np.unique(np.diff(data['packetID']))
assert(len(jumps) and jumps[0] == 1)

#### Save data .mat file

In [10]:
decoded = {"analog":data['analog'], "digitalIn":data['digital_in'], "digitalOut":data['digital_out'], "startTS":data['us_start'], "transmitTS":data['us_end'], "longVar":data['states'], "packetNums":data['packetID']}
path=os.path.join(Path_decoded, Name+'_decoded'+'.mat')
savemat(path, decoded)

### Post decoding processing to extract time stamps in frames for analysis

In [71]:
packet_time_stamp = data['us_start']

scannerDat = scipy.io.loadmat('/home/melisamc/Documentos/photon2_testing/20220323_audseq_00002_scanner.mat')

In [105]:
frameTs = np.zeros((len(scannerDat['frameTs']),))
for i in range(len(scannerDat['frameTs'])):
    frameTs[i] = scannerDat['frameTs'][i]
print('Number of frames:', frameTs.shape)
print('Mean Scanner time per frame:', np.mean(np.diff(frameTs)))

Number of frames: (56326,)
Mean Scanner time per frame: 0.0333949061831336


In [88]:
### compute offset assuming scanner time starts in 0
x = decoded['digitalIn'][:,0]
print(x[0])
x_diff = np.diff(x)
positions = np.where(x_diff == 1)[0]+1
what_we_want = x_diff[positions]

sampling_rate_scanner = np.mean(np.diff(positions))
offset_frames = round(positions[0]*sampling_rate_scanner/1000)

print('Offset is: ' + f'{positions[0]}' + ' ms ' )
print('Offset is: ' + f'{offset_frames}' + ' frames ')

print('Number of frames is: ', len(what_we_want))
print('Instantaneous Sampling rate is: ', sampling_rate_scanner)

0
Offset is: 21622 ms 
Offset is: 722 frames 
Number of frames is:  56325
Instantaneous Sampling rate is:  33.394592003408846


In [123]:
### sound 1

#get sound signal from digital inputs
sound1 = decoded['digitalIn'][:,8]
sound2 = decoded['digitalIn'][:,9]
sound3 = decoded['digitalIn'][:,10]
sound4 = decoded['digitalIn'][:,11]
sound5 = decoded['digitalIn'][:,12]
sound6 = decoded['digitalIn'][:,13]
iti = decoded['digitalIn'][:,14]

#get only the transientes for obtening the onsets (this is in packect number ID)
sound1_diff = np.diff(sound1)
sound2_diff = np.diff(sound2)
sound3_diff = np.diff(sound3)
sound4_diff = np.diff(sound4)
sound5_diff = np.diff(sound5)
sound6_diff = np.diff(sound6)
iti_diff = np.diff(iti)

## get packet stamps in index from packets
pack_stamps_sound1 = np.where(sound1_diff == 1)[0]
pack_stamps_sound2 = np.where(sound2_diff == 1)[0]
pack_stamps_sound3 = np.where(sound3_diff == 1)[0]
pack_stamps_sound4 = np.where(sound4_diff == 1)[0]
pack_stamps_sound5 = np.where(sound5_diff == 1)[0]
pack_stamps_sound6 = np.where(sound6_diff == 1)[0]
pack_stamps_iti = np.where(iti_diff == 1)[0]-1

#convert packet number ID into time_stamps in micro seconds
# time_stamp_sound1 = ((packet_time_stamp[pack_stamps_sound1] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_sound2 = ((packet_time_stamp[pack_stamps_sound2] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_sound3 = ((packet_time_stamp[pack_stamps_sound3] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_sound4 = ((packet_time_stamp[pack_stamps_sound4] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_sound5 = ((packet_time_stamp[pack_stamps_sound5] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_sound6 = ((packet_time_stamp[pack_stamps_sound6] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames
# time_stamp_iti = ((packet_time_stamp[pack_stamps_iti] - packet_time_stamp[0])*sampling_rate_scanner/10**6).astype(int) - offset_frames


sound1_onset = ((pack_stamps_sound1 - positions[0])/1000)
[values,edges] = np.histogram(sound1_onset,bins = frameTs)
time_stamp_sound1 = np.where(values==1)[0]

sound2_onset = ((pack_stamps_sound2 - positions[0])/1000)
[values,edges] = np.histogram(sound2_onset,bins = frameTs)
time_stamp_sound2 = np.where(values==1)[0]

sound3_onset = ((pack_stamps_sound3 - positions[0])/1000)
[values,edges] = np.histogram(sound3_onset,bins = frameTs)
time_stamp_sound3 = np.where(values==1)[0]

sound4_onset = ((pack_stamps_sound4 - positions[0])/1000)
[values,edges] = np.histogram(sound4_onset,bins = frameTs)
time_stamp_sound4 = np.where(values==1)[0]

sound5_onset = ((pack_stamps_sound5 - positions[0])/1000)
[values,edges] = np.histogram(sound5_onset,bins = frameTs)
time_stamp_sound5 = np.where(values==1)[0]

sound6_onset = ((pack_stamps_sound6 - positions[0])/1000)
[values,edges] = np.histogram(sound6_onset,bins = frameTs)
time_stamp_sound6 = np.where(values==1)[0]

iti_onset = ((pack_stamps_iti - positions[0])/1000)
[values,edges] = np.histogram(iti_onset,bins = frameTs)
time_stamp_iti = np.where(values==1)[0]



#verify whether there is any packet loss
# loss = np.mean((pack_stamps_sound1*1000 - time_stamp_sound1)**2) + np.mean((pack_stamps_sound2*1000 - time_stamp_sound2)**2) 
# + np.mean((pack_stamps_sound3*1000 - time_stamp_sound3)**2) + np.mean((pack_stamps_sound4*1000 - time_stamp_sound4)**2) 
# + np.mean((pack_stamps_sound5*1000 - time_stamp_sound5)**2) + np.mean((pack_stamps_sound6*1000 - time_stamp_sound6)**2) 
# + np.mean((pack_stamps_iti*1000 - time_stamp_iti)**2) 

print('Number of lost packets :' , loss)


Number of lost packets : 1.9277518912124692e+18


In [124]:
output = {'sampling_rate': sampling_rate_scanner,'offset': offset_frames,'sound1':time_stamp_sound1, 'sound2': time_stamp_sound2, 
          'sound3': time_stamp_sound3, 'sound4': time_stamp_sound4, 'sound5': time_stamp_sound5, 'sound6': time_stamp_sound6, 'iti': time_stamp_iti}
path=os.path.join(path_output, Name+'_output'+'.pickle')
#np.save(path,output)

In [125]:
time_stamp_sound1

array([ 6996,  7717,  9519, 10241, 14926, 18536, 18897, 19257, 22141,
       25748, 26469, 27552, 29535, 31335, 38185, 38546, 41788, 42149,
       44310, 46112])

In [126]:
with open(path, 'wb') as f:
    pickle.dump(output, f)

In [127]:
output = {'sampling_rate': sampling_rate_scanner,'offset': offset_frames,'sound1':time_stamp_sound1, 'sound2': time_stamp_sound2, 
          'sound3': time_stamp_sound3, 'sound4': time_stamp_sound4, 'sound5': time_stamp_sound5, 'sound6': time_stamp_sound6, 'iti': time_stamp_iti}
path=os.path.join(path_output, Name+'_output'+'.mat')
savemat(path, output)

In [128]:
offset_frames

722

In [82]:
edges

array([0.00000000e+00, 3.33845250e-02, 6.67690750e-02, ...,
       1.88090130e+03, 1.88093470e+03, 1.88096809e+03])

In [38]:
pack_stamps_sound1*sampling_rate_scanner/1000 - offset_frames

array([ 7802.37034397,  8606.04459513, 10615.73114189, 11420.8413605 ,
       16646.29353883, 20672.24536698, 21074.18267634, 21476.28695865,
       24692.88745501, 28715.06569427, 29519.14068053, 30726.75591655,
       32937.84524769, 34945.82867026, 42585.27572074, 42987.27981928,
       46603.51339814, 47005.08336698, 49415.33804483, 51425.12477537])

In [67]:
sound1_onset = ((pack_stamps_sound1 - positions[0])/1000)

In [84]:
sound1_onset

array([ 233.64 ,  257.706,  317.886,  341.995,  498.471,  619.028,
        631.064,  643.105,  739.426,  859.87 ,  883.948,  920.11 ,
        986.321, 1046.45 , 1275.213, 1287.251, 1395.539, 1407.564,
       1479.739, 1539.922])

In [108]:
frameTs

array([0.00000000e+00, 3.33845250e-02, 6.67690750e-02, ...,
       1.88090130e+03, 1.88093470e+03, 1.88096809e+03])